In [2]:
import pandas as pd

df_1 = pd.read_csv("filtered_data_197971.csv", index_col=0)
features = ["SALE TIMES", "GROSS LAND RATIO", 'DAY','MONTH','YEAR',
            'BUILDING CLASS CATEGORY', 'ZIP CODE', 'RESIDENTIAL UNITS',
            'COMMERCIAL UNITS', 'LAND SQUARE FEET', 'GROSS SQUARE FEET',
            "UPPER BUILDING CLASS AT TIME OF SALE",
            'AGE', 'TAX CLASS AT TIME OF SALE', 'BUILDING CLASS AT TIME OF SALE',
            'Latitude', 'Longitude',"SALE PRICE"]
df_1['SALE DATE'] = pd.to_datetime(df_1['SALE DATE'])
df_1['DAY'] = df_1['SALE DATE'].dt.day
df_1['MONTH'] = df_1['SALE DATE'].dt.month
df_1['YEAR'] = df_1['SALE DATE'].dt.year
df_1["SALE TIMES"] = df_1["ADDRESS"].map(df_1["ADDRESS"].value_counts().to_dict())
df_1["GROSS LAND RATIO"] = df_1["GROSS SQUARE FEET"]/df_1["LAND SQUARE FEET"]
df_1["UPPER BUILDING CLASS AT TIME OF SALE"] = df_1[f"BUILDING CLASS AT TIME OF SALE"].str[0]
df_1 = df_1[features]

In [8]:
import h2o
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators import H2ORandomForestEstimator
from h2o.frame import H2OFrame
import numpy as np
from itertools import chain
import matplotlib.pyplot as plt
import seaborn as sns
import time

def get_percentage_error(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred) / np.abs(y_true) * 100)

def get_df_dict(df_1,by,category):
    category_dict = {"residential": ["A","B","C","D",'L','R',"S"],
                    "house": ["A","B"],
                    "apartment":["C","D",'L','R',"S"],
                    "commercial":['I','H','J','K','O'],
                    "all":[chr(ord('a') + i) for i in range(26)]}
    month_mapping = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr',
                     5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug',
                     9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}
    df_dict = {}
    df_1 = df_1[df_1["BUILDING CLASS AT TIME OF SALE"].apply(lambda x: x[0] in category_dict[category])]
    if by == "month":
        for year in sorted(list(set(df_1["YEAR"]))):
            for month in sorted(list(set(df_1["MONTH"]))):
                df = df_1[(df_1["YEAR"]==year)&(df_1["MONTH"]==month)]
                df_dict[" ".join([str(year),month_mapping[month]])] = df
    if by == "year":
        for year in sorted(list(set(df_1["YEAR"]))):
                df = df_1[df_1["YEAR"]==year]
                df_dict[str(year)] = df
    return df_dict

def rf_get_result_cv(df, target):
    print(df.shape)
    h2o_df = H2OFrame(df)
    response_col = target
    predictors = [col for col in h2o_df.columns if col != response_col]
    train, test = h2o_df.split_frame(ratios=[0.9], seed=42)
    rf_model = H2ORandomForestEstimator(
        seed=42,
        nfolds=5,
        keep_cross_validation_predictions=True
    )
    hyperparameters = {
        'ntrees': [10, 20, 30, 50, 100, 150],
        'max_depth': [1, 2, 3, 5, 10, 20, 30, 50]
    }
    grid = H2OGridSearch(rf_model, hyperparameters)
    grid.train(x=predictors, y=response_col, training_frame=train)
    rf_model = grid.get_grid()[0]
    feature_importance = rf_model.varimp()
    predictions = rf_model.predict(test).as_data_frame().values
    test_response = test.as_data_frame()[response_col].values
    mse = rf_model.mse()
    mae = rf_model.mae()
    percentage_error = get_percentage_error(test_response,predictions)
    print("mse: ",mse,"\n","mae: ",mae,"\n","percentage_error: ",percentage_error)
    return (mse,mae,percentage_error,feature_importance)

def expermient(df_1,by,category,target):
    mse_list = []
    mae_list = []
    percentage_error_list = []
    feature_importance_list = []
    df_dict = get_df_dict(df_1,by,category)
    h2o.init()
    for key,val in list(df_dict.items())[:]:
        print(key,"  start")
        start_time = time.time()
        mse,mae,percentage_error,feature_importance = rf_get_result_cv(val, target)
        mse_list.append(mse)
        mae_list.append(mae)
        percentage_error_list.append(percentage_error)
        feature_importance_list.append(feature_importance)
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(key,"  end","\n\n")
        print(f"Time: {elapsed_time} seconds")
    return df_dict, mse_list, mae_list, percentage_error_list, feature_importance_list

def draw_feature_importance_map(df_dict,feature_importance_list,category,target,by):
    flat_feature_importance = [item for sublist in feature_importance_list for item in sublist]
    df = pd.DataFrame(flat_feature_importance, columns=['Feature', 'Importance', 'Relative Importance', 'Cumulative Importance'])
    feature_num = df_dict[list(df_dict.keys())[0]]-1
    df['Year'] = list(chain(*[[key]*feature_num for key in df_dict.keys()]))
    df_pivot = df.pivot_table(index='Feature', columns='Year', values='Cumulative Importance', aggfunc='first')
    plt.figure(figsize=(12, 8))
    sns.heatmap(df_pivot, annot=True, cmap='Blues', fmt=".2%", cbar_kws={'label': 'Relative Importance'})
    plt.title(title:=f'Feature Importance Heatmap of {target.title()} of {category.title()} Buildings by {by}')
    plt.xlabel('Year')
    plt.ylabel('Feature')
    plt.savefig("../plots_2/"+title)
    plt.show()

In [9]:
import pandas as pd

df_1 = pd.read_csv("filtered_data_197971.csv", index_col=0)
features = ["SALE TIMES", "GROSS LAND RATIO", 'DAY','MONTH','YEAR',
            'BUILDING CLASS CATEGORY', 'ZIP CODE', 'RESIDENTIAL UNITS',
            'COMMERCIAL UNITS', 'LAND SQUARE FEET', 'GROSS SQUARE FEET',
            "UPPER BUILDING CLASS AT TIME OF SALE",
            'AGE', 'TAX CLASS AT TIME OF SALE', 'BUILDING CLASS AT TIME OF SALE',
            'Latitude', 'Longitude',"SALE PRICE"]
df_1['SALE DATE'] = pd.to_datetime(df_1['SALE DATE'])
df_1['DAY'] = df_1['SALE DATE'].dt.day
df_1['MONTH'] = df_1['SALE DATE'].dt.month
df_1['YEAR'] = df_1['SALE DATE'].dt.year
df_1["SALE TIMES"] = df_1["ADDRESS"].map(df_1["ADDRESS"].value_counts().to_dict())
df_1["GROSS LAND RATIO"] = df_1["GROSS SQUARE FEET"]/df_1["LAND SQUARE FEET"]
df_1["UPPER BUILDING CLASS AT TIME OF SALE"] = df_1[f"BUILDING CLASS AT TIME OF SALE"].str[0]
df_1 = df_1[features]

category,target,by = "house","PRICE PER GROSS AREA","year"
df_dict, mse_list, mae_list, percentage_error_list, feature_importance_list = expermient(df_1,by,category,target)
draw_feature_importance_map(df_dict,feature_importance_list,category,target,by)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,2 mins 00 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,1 month
H2O_cluster_name:,H2O_from_python_DELL_971br6
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.933 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


2016   start
(21537, 18)
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


TypeError: list indices must be integers or slices, not str